In [4]:
!pip install pandas
!pip install numpy


  Using cached pandas-1.5.1-cp38-cp38-macosx_11_0_arm64.whl (10.7 MB)


In [5]:
import pandas as pd
import numpy as np

import tensorflow as tf

NotFoundError: dlopen(/Users/ecemdenizbabaoglan/opt/anaconda3/envs/tf/lib/python3.8/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 6): Symbol not found: __ZNKSt3__115basic_stringbufIcNS_11char_traitsIcEENS_9allocatorIcEEE3strEv
  Referenced from: /Users/ecemdenizbabaoglan/opt/anaconda3/envs/tf/lib/python3.8/site-packages/tensorflow-plugins/libmetal_plugin.dylib (which was built for Mac OS X 12.3)
  Expected in: /usr/lib/libc++.1.dylib


In [ ]:
!conda install -c apple tensorflow-deps -y

In [2]:
pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 2.0 MB/s eta 0:00:01     |██████████████████████████▏     | 1.7 MB 1.1 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np

import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

NotFoundError: dlopen(/Users/ecemdenizbabaoglan/opt/anaconda3/envs/tf/lib/python3.8/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 6): Symbol not found: __ZNKSt3__115basic_stringbufIcNS_11char_traitsIcEENS_9allocatorIcEEE3strEv
  Referenced from: /Users/ecemdenizbabaoglan/opt/anaconda3/envs/tf/lib/python3.8/site-packages/tensorflow-plugins/libmetal_plugin.dylib (which was built for Mac OS X 12.3)
  Expected in: /usr/lib/libc++.1.dylib


In [ ]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [ ]:
dataset = pd.read_csv("/Users/ecemdenizbabaoglan/yap470-proje/new_df.csv")

In [ ]:
labels = dataset['Label']
data = dataset.loc[:, dataset.columns != 'Label']

In [ ]:
labels.value_counts()

In [ ]:
labels.loc[labels == 'Benign'].shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size = 0.2, random_state = 12345, stratify = labels)

In [ ]:
y_train.value_counts()

In [ ]:
benign_train_data = x_train.loc[y_train == 'Benign']
attack_train_data = x_train.loc[y_train != 'Benign']
benign_test_data = x_test.loc[y_test == 'Benign']
attack_test_data = x_test.loc[y_test != 'Benign']

In [ ]:
attack_train_data.shape

In [ ]:
y_train.loc[y_train != 'Benign'] = 1
y_train.loc[y_train == 'Benign'] = 2
y_test.loc[y_test != 'Benign'] = 1
y_test.loc[y_test == 'Benign'] = 2

In [ ]:
x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

x_train = tf.cast(x_train, tf.float32)
x_test = tf.cast(x_test, tf.float32)

In [ ]:
benign_train_data.shape

In [ ]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(32, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(140, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [ ]:
autoencoder.compile(optimizer='adam', loss='mae')

In [ ]:
history = autoencoder.fit(benign_train_data, benign_train_data, 
          epochs=20, 
          batch_size=512,
          validation_data=(x_test, x_test),
          shuffle=True)